# Tracking Article Processing Charges (APCs) for a given institution

In this notebook, we will query the OpenAlex API to answer the following questions:  

1. **How much are researchers at my institution paying in APCs?**
2. **Which journals/publishers are collecting the most APCs from researchers at my institution?**
3. **How much money are my organization’s researchers saving in discounted APC charges from our transformative/read-publish agreements?**

Most organizations do not have an effective way of tracking the APCs that their researchers pay to publish in open access journals.  By estimating how much money is going to APCs each year, and which publishers are collecting the most APCs, libraries can make more informed decisions around the details of the read-publish agreements they have with various publishers.  

### APC-able Works

Before starting this analysis, it is important to define which types of works are subject to APCs and which are not.   

While a work may include contributions from a number of different institutions, the APC is typically the responsbility of the work's *corresponding author*.  

In addition, open access works published in Gold and Hybrid OA journals are subject to APCs, while those published in Green, Diamond, and Bronze OA journals are not.  

Finally, APCs are not typically charged for editorial content submitted to an open access journal.  

Thus, for the purposes of this notebook, *APC-able works* must have the following characteristics:
- Original articles or reviews
- Published in a Gold or Hybrid OA journal
- Corresponding author is a researcher at our institution.

## Surveying APCs by journal/publisher

### Steps

1. We need to get all the works published and corresponded by researchers at the institution
2. We get the journal/publisher and APC for each publication
3. We sum the APCs (by journal/publisher)

### Input

For inputs, we first need to identify the Research Organization Registry (ROR) ID for our institution. In this example we will use the ROR ID for McMaster University ([https://ror.org/02fa3aq29](https://ror.org/02fa3aq29)). You can search and substitute your own institution's ROR here: [https://ror.org/search](https://ror.org/search).  

Next, we identify the publication year we are interested in analyzing. If the details of your institution's specific tranformative/read-publish agreements change from year to year, you will want to limit your analysis to a single year.  

Finally, becauase editorial content is not typically subject to APCs, we will limit our search to works with the publication types "article" or "review".  

In [ ]:
RUN_OPTIONAL = True  # flag to determine whether to run optional code
SAVE_CSV = True  # flag to determine whether to save the output as a CSV file
FIRST_NUM_ROWS = 10  # number of top items to display

# input
ror_ids = [
    "https://ror.org/02fa3aq29",  # McMaster University
    "https://ror.org/03cegwq60",  # McMaster Children's Hospital
    "https://ror.org/02zrdtc90",  # McMaster Divintiy College
    "https://ror.org/05jyrng31",  # McMaster University Medical Center
]
publication_year = 2024
publication_types = ["article", "review"]
publication_oa_statuses = ["gold", "hybrid"]

### Get OpenAlex ID of the given institution

We only want publications with corresponding authors, who are affiliated with McMaster University. However, OpenAlex currently does not support filtering corresponding institutions by ROR ID, we will need to find out the OpenAlex ID for McMaster using the [`institutions`](https://docs.openalex.org/api-entities/institutions) entity type.  

Our search criteria are as follows:  
- `ror`: ROR ID of the institution, `ror:https://ror.org/02fa3aq2`

Now we need to build an URL for the query from the following parameters:  
- Starting point is the base URL of the OpenAlex API: `https://api.openalex.org/`
- We append the entity type to it: `https://api.openalex.org/institutions`
- All criteria need to go into the query parameter filter that is added after a question mark: `https://api.openalex.org/institutions?filter=`
- To construct the filter value we take the criteria we specified and concatenate them using commas as separators: `https://api.openalex.org/institutions?filter=ror:https://ror.org/02fa3aq29`

```py
# construct the url using the provided ror id
url = f"https://api.openalex.org/institutions?filter=ror:{ror_id}"

# send a get request to the constructed url
response = requests.get(url)

# parse the response json data
json_data = response.json()

# extract the institution id from the first result
institution_id = json_data["results"][0]["id"]
```

In [ ]:
from openalex_helpers.institutions import get_institution_by_ror

institution_ids = [get_institution_by_ror(ror_id)["id"] for ror_id in ror_ids]
institution_ids

### Get all APC-able works published by researchers at the institution

Our search criteria are as follows:  
- `corresponding_institution_ids`: institution affiliated with the corresponding authors of a work (OpenAlex ID), `corresponding_institution_ids:https://openalex.org/I98251732|https://openalex.org/I2801880889|https://openalex.org/I163387037|https://openalex.org/I2801259263`
- `publication_year`: the year the work was published, `publication_year:2024`
- [`types`](https://docs.openalex.org/api-entities/works/work-object#type): the type of the work, `type:article|review`
- [`oa_status`](https://docs.openalex.org/api-entities/works/work-object#open_access): the OA status of the work, `oa_status:gold|hybrid`

Now we need to build an URL for the query from the following parameters:  
- Starting point is the base URL of the OpenAlex API: `https://api.openalex.org/`
- We append the entity type to it: `https://api.openalex.org/works`
- All criteria need to go into the query parameter filter that is added after a question mark: `https://api.openalex.org/works?filter=`
- To construct the filter value we take the criteria we specified and concatenate them using commas as separators: `https://api.openalex.org/works?filter=corresponding_institution_ids:https://openalex.org/I98251732,publication_year:2024,type:article|review,oa_status:gold|hybrid&page=1&per-page=50`


```py
def get_works_by_corresponding_institutions(institution_ids, publication_year, publication_types, page=1, items_per_page=50):
    # construct the api url with the given institution ids, publication year, publication types, page number, and items per page
    url = f"https://api.openalex.org/works?filter=corresponding_institution_ids:{'|'.join(institution_ids)},publication_year:{publication_year},type:{'|'.join(publication_types)},oa_status:{'|'.join(publication_oa_statuses)}&page={page}&per-page={items_per_page}"

    # send a GET request to the api and parse the json response
    response = requests.get(url)
    json_data = response.json()

    # convert the json response to a dataframe
    df_json = pd.DataFrame.from_dict(json_data["results"])

    next_page = True
    if df_json.empty: # check if the dataframe is empty (i.e., no more pages available)
        next_page = False

    # if there are more pages, recursively fetch the next page
    if next_page:
        df_json_next_page = get_works_by_corresponding_institutions(institution_ids, publication_year, publication_types, page=page+1, items_per_page=items_per_page)
        df_json = pd.concat([df_json, df_json_next_page])

    return df_json
```

In [ ]:
import pandas as pd
from openalex_helpers.works import get_works_by_corresponding_institutions

try:
    from helpers.converters import to_dict_convertor
    # read the institution_works_{publication_year}.csv file if it exists, otherwise query data from OpenAlex
    df_works = pd.read_csv(f"data/apc/institution_works_{publication_year}.csv", converters={ "apc_list": to_dict_convertor, "primary_location": to_dict_convertor })  # use converters to parse 'apc_list' and 'primary_location' columns as dictionaries
except FileNotFoundError:
    df_works = get_works_by_corresponding_institutions(
        institution_ids, publication_year, publication_types, publication_oa_statuses
    )

    if SAVE_CSV:
        df_works.to_csv(f"data/apc/institution_works_{publication_year}.csv", index=False)

df_works

### Get Journals/Publishers and APCs in USD

In a `work` entity object, there is information about the journal (`primary_location`) and the journal's APC list price ([`apc_list`](https://docs.openalex.org/api-entities/works/work-object#apc_list)).  

is derived from the [Directory of Open Access Journals (DOAJ)](https://doaj.org/), which compiles APC data currently available on publishers' websites.  

It should be noted that not all publishers list APC prices on their websites, meaning that not all works will have an `apc_list` price in OpenAlex.  In these cases, we will infer the APC price based on the mean APC prices of those works for which `apc_list` data is available.  

In addition, even when APC price is included on a publishers' website, there is no guarantee that this is the final APC price our authors paid for publication.  

For these reasons, results of this notebook must be understood as best available estimates.  

In [ ]:
import numpy as np

# extract 'value_usd' from 'apc_list' if it is a dictionary (i.e. 'apc_list' exists in the work record); otherwise, set to null
df_works["apc_list_usd"] = df_works["apc_list"].apply(lambda apc_list: apc_list["value_usd"] if isinstance(apc_list, dict) else np.nan)

# extract 'id' and 'name' from 'source' within 'primary_location' if 'source' exists; otherwise, set to null
df_works["source_id"] = df_works["primary_location"].apply(lambda location: location["source"]["id"] if location["source"] else np.nan)
df_works["source_name"] = df_works["primary_location"].apply(lambda location: location["source"]["display_name"] if location["source"] else np.nan)

# extract 'host_organization' from 'source' within 'primary_location' if 'source' exists; otherwise, set to null
df_works["source_host_organization"] = df_works["primary_location"].apply(lambda location: location["source"]["host_organization"] if location["source"] else np.nan)

# extract 'issn' and 'issn_l' from 'source' within 'primary_location' if 'source' exists; otherwise, set to null
df_works["source_issn"] = df_works["primary_location"].apply(lambda location: location["source"]["issn"] if location["source"] else np.nan)
df_works["source_issn_l"] = df_works["primary_location"].apply(lambda location: location["source"]["issn_l"] if location["source"] else np.nan)

In [ ]:
# calculate the average apc where 'apc_list_usd' is not null
apc_mean = df_works[df_works["apc_list_usd"].notnull()]["apc_list_usd"].mean()

# fill null values in 'apc_list_usd' with the calculated average
df_works["apc_list_usd"] = df_works["apc_list_usd"].fillna(apc_mean)

# fill null values in 'source_id', 'source_name', 'source_issn' and 'source_issn_l'
df_works["source_id"] = df_works["source_id"].fillna("unknown source")
df_works["source_name"] = df_works["source_name"].fillna("unknown source")
df_works["source_host_organization"] = df_works["source_host_organization"].fillna("unknown source")
df_works["source_issn"] = df_works["source_issn"].fillna("unknown source")
df_works["source_issn_l"] = df_works["source_issn_l"].fillna("unknown source")

### Get Publisher Display Name (Optional)

OpenAlex identifies publishers with a unique identfier called an OpenAlex ID. The following code translates this OpenAlex ID to the publisher's display name for easier analysis.  

In [ ]:
import re
import requests

CHUNK_SIZE = 5

def get_source_host_organization_display_name(publisher_ids):
    def get_source_host_organization_publisher_display_name(publisher_ids):
        def get_source_host_organization_publisher_display_name_by_chunk(publisher_ids_chunk):
            # construct the api url using the chunk of publisher ids
            url = f"https://api.openalex.org/publishers?filter=ids.openalex:{'|'.join(publisher_ids_chunk)}"

            # send a GET request to the api and parse the json response
            response = requests.get(url)
            json_data = response.json()

            # convert the json response to a dataframe and return the relevant columns
            df_json = pd.DataFrame.from_dict(json_data["results"])
            return df_json[["id", "display_name"]]
        
        # check if the length of 'publisher_ids' is less than 1
        if len(publisher_ids) < 1:
            # if true, return an empty dataframe
            return pd.DataFrame()

        # split the publisher ids into chunks and apply the function to each chunk
        chunks = np.array_split(publisher_ids, np.ceil(len(publisher_ids) / CHUNK_SIZE))
        df_chunks = pd.DataFrame({"chunk": chunks})
        return pd.concat(df_chunks["chunk"].apply(get_source_host_organization_publisher_display_name_by_chunk).tolist())

    def get_source_host_organization_institution_display_name(institution_ids):
        def get_source_host_organization_institution_display_name_by_chunk(institution_ids_chunk):
            # construct the api url using the chunk of institution ids
            url = f"https://api.openalex.org/institutions?filter=id:{'|'.join(institution_ids_chunk)}"

            # send a GET request to the api and parse the json response
            response = requests.get(url)
            json_data = response.json()

            # convert the json response to a dataframe and return the relevant columns
            df_json = pd.DataFrame.from_dict(json_data["results"])
            return df_json[["id", "display_name"]]

        # check if the length of 'institution_ids' is less than 1
        if len(institution_ids) < 1:
            # if true, return an empty dataframe
            return pd.DataFrame()

        # split the institution ids into chunks and apply the function to each chunk
        chunks = np.array_split(institution_ids, np.ceil(len(institution_ids) / CHUNK_SIZE))
        df_chunks = pd.DataFrame({"chunk": chunks})
        return pd.concat(df_chunks["chunk"].apply(get_source_host_organization_institution_display_name_by_chunk).tolist())

     # filter the publisher ids to get only publisher urls
    publishers = list(filter(lambda s: re.search(r"https:\/\/openalex\.org\/P", s), publisher_ids))
    # filter the institution ids to get only institution urls
    institutions = list(filter(lambda s: re.search(r"https:\/\/openalex\.org\/I", s), publisher_ids))

    # create a dataframe with a default entry for unknown source
    df_lookup = pd.DataFrame.from_dict({"id": ["unknown source"], "display_name": ["unknown source"]})
    # concatenate the dataframes with publisher and institution display names
    df_lookup = pd.concat([df_lookup, get_source_host_organization_publisher_display_name(publishers), get_source_host_organization_institution_display_name(institutions)], ignore_index=True)
    return df_lookup

In [ ]:
if RUN_OPTIONAL:
    # read the source_host_organization_lookup.csv file if it exists, otherwise query data from OpenAlex
    try:
        df_lookup = pd.read_csv(f"data/apc/source_host_organization_lookup.csv")
    except FileNotFoundError:
        # get the display names for unique source_host_organization ids in df_works
        df_lookup = get_source_host_organization_display_name(df_works["source_host_organization"].unique())
        if SAVE_CSV:
            df_lookup.to_csv(f"data/apc/source_host_organization_lookup.csv", index=False)

In [ ]:
if RUN_OPTIONAL:
    # update the 'source_host_organization' with the corresponding display names
    df_works["source_host_organization"] = df_works["source_host_organization"].apply(lambda publisher: df_lookup[df_lookup["id"] == publisher]["display_name"].squeeze())  

### Get Journals/Publishers and APCs in USD (Continued)

In [ ]:
# rename the 'apc_list_usd' column to 'apc_usd'
df_apc = df_works.rename(columns={"apc_list_usd": "apc_usd"})
if SAVE_CSV:
    df_apc.to_csv(f"data/apc/apc_usd.csv", index=False)

df_apc

### Aggregate APCs Data (Optional)

Here, we build a dataframe containing the number of APC-able works and the estiamted total APC cost for each journal.

In [ ]:
if RUN_OPTIONAL:
    # group the dataframe by 'source_id' and 'source_issn_l'
    # and aggregate 'source_issn' by taking the maximum value (in this case the common issn list of strings)
    # and 'source_host_organization' by taking the maximum value (in this case the common string name of the source's host organization)
    # and 'source_name' by taking the maximum value (in this case the common string name of the source)
    # and 'id' by counting
    # and 'apc_list_usd' by summing
    df_apc = df_works.groupby(["source_id", "source_issn_l"]).agg({"source_issn": "max", "source_name": "max", "source_host_organization": "max", "id": "count", "apc_list_usd": "sum"})
    # rename the 'id' column to 'num_publications' and 'apc_list_usd' column to 'apc_usd'
    df_apc.rename(columns={"id": "num_publications", "apc_list_usd": "apc_usd"}, inplace=True)
    if SAVE_CSV:
        df_apc.to_csv(f"data/apc/apc_usd_by_source.csv", index=False)

### Estimating the total (non-discounted) APC spend

In [ ]:
total_apc = df_apc["apc_usd"].sum()
print(f"Estimated total (non-discounted) APC cost in {publication_year}: USD {round(total_apc, 2)}.")

## Calculating Discounted APCs

### Steps

1. We load the given list of read-publish agreement discounts
2. We check if publishers are included in the list by ISSN
3. We calculate the APCs paid with the list of read-publish agreement discounts and the APC listed prior

### Input

Assume we have list of read-publish agreement discounts in CSV format, `discount-list.csv`. In the file, it includes the following necessary attributes,  
- `issn`: ISSN of the publisher
- `discount`: value of the discount, either a number or a percentage
- `is_flatrate`: flag indicating whether the discount is a flat rate discount or a percentage discount

You can download a template [`data/apc/discount-list.csv`](data/apc/discount-list.csv.example) here and update it with the details of your institutions own APC discounts.  

In [ ]:
# input
df_discount = pd.read_csv("data/apc/discount-list.csv")

In [ ]:
import typing

def get_discount(issn: typing.List[str] | str, apc: float, num_publications: typing.Optional[int]) -> float:
    # check if issn is a string, if so, convert it to a list
    if isinstance(issn, str):
        issn = [issn]

    # filter the discount dataframe to get rows where 'issn' is in the provided issn list
    discount_rows = df_discount[df_discount["issn"].isin(issn)]
    
    # if no discount rows are found, return the original apc
    if discount_rows.empty:
        return apc

    # get the first row from the filtered discount rows
    discount_row = discount_rows.iloc[0]
    
    # if the discount is a flat rate, subtract the discount from the apc
    if discount_row["is_flatrate"]:
        return apc - discount_row["discount"] * (num_publications if num_publications is not None else 1)  # discount is applied per publication
    else:
        # if the discount is a percentage, apply the discount to the apc
        return apc * (1 - discount_row["discount"])

### Apply Discounts to APC Data

Here, we apply the APC discounts to the APC data. This produces a dataframe and `.csv` file that includes the number of APC-able publications and the discounted APC cost for each journal.  

In [ ]:
# apply the get_discount function to each row of the dataframe to calculate the discounted apc and store it in a new column 'discounted_apc_usd'
df_apc["discounted_apc_usd"] = df_apc.apply(lambda x: get_discount(issn=x["source_issn"], apc=x["apc_usd"], num_publications=(x["num_publications"] if "num_publications" in df_apc.columns else None)), axis=1)
if SAVE_CSV:
    df_apc.to_csv(f"data/apc/apc_usd_with_discounts.csv", index=False)

df_apc

### Graph Top Publishers by APC Saved (Optional)

In [ ]:
if RUN_OPTIONAL:
    import matplotlib.pyplot as plt

    # group the dataframe by 'source_host_organization'
    # and aggregate 'num_publications' by summing
    # and 'apc_usd' by summing
    # and 'discounted_apc_usd' by summing
    _df_apc = df_apc.groupby(["source_host_organization"]).agg({"num_publications": "sum", "apc_usd": "sum", "discounted_apc_usd": "sum"})
    # calculate the total apc savings by subtracting discounted apc from original apc
    _df_apc["apc_saved"] = _df_apc["apc_usd"] - _df_apc["discounted_apc_usd"]

    if SAVE_CSV:
        _df_apc.to_csv(f"data/apc/apc_usd_with_discounts_by_source_host_organization.csv", index=False)

    # sort the dataframe by 'discounted_apc_usd' in descending order and select the top rows
    _df_apc = _df_apc.sort_values(by="apc_saved", ascending=False).head(FIRST_NUM_ROWS)

    # create the figure and the first y-axis
    fig, ax1 = plt.subplots()
    # create a second y-axis that shares the same x-axis
    ax2 = ax1.twinx()

    # define the width of each bar and the positions for the bars
    bar_width = 0.35
    positions = np.arange(len(_df_apc))

    # plot the bars for discounted apc on the first y-axis
    bar1 = ax1.bar(positions - bar_width / 2, _df_apc["apc_saved"], bar_width, label="Discounted APC (USD)")
    # plot the bars for number of publications on the second y-axis
    bar2 = ax2.bar(positions + bar_width / 2, _df_apc["num_publications"], bar_width, color="orange", label="Number of Publications")

    # set the x-axis tick positions and labels
    ax1.set_xticks(positions)
    ax1.set_xticklabels(_df_apc.index, rotation="vertical")

    # set the x-axis label
    ax1.set_xlabel("Publisher")

    # set the y-axis labels for both axes
    ax1.set_ylabel("APC Saved (USD)")
    ax2.set_ylabel("Number of Publications")

    # add a combined legend for both bar plots
    ax1.legend([bar1, bar2], [bar1.get_label(), bar2.get_label()], loc="upper right")

    # add value labels on top of the bars for number of publications
    for bar in bar2:
        yval = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width() / 2, yval, int(yval), ha="center", va="bottom")

    # set the title of the plot
    plt.title(f"Top {FIRST_NUM_ROWS} publishers by APC saved in {publication_year}")

    plt.show()

### Estimating the total discounted APC cost

In [ ]:
total_apc_discount = df_apc["discounted_apc_usd"].sum()
print(f"Estimated APC cost (including discounts) for {publication_year}: USD {round(total_apc_discount, 2)}.")

### Estimating the total APC savings of our institution's read-publish agreements

In [ ]:
print(f"Estimated APC saving in {publication_year}: USD {round(total_apc - total_apc_discount, 2)}.")